# Pricer Demo
This notebook serves the purpose of understanding the example file found at https://github.com/scipopt/PySCIPOpt/blob/use-probdata-to-store-model/tests/test_coloring.py

I guess this is the idea of everything:
The formulation is:  $$\min{\sum_{s \in S} x_s :  \sum_{s \in S: u \in s} x_s \geq 1 \text{ for all } u \in V}$$
where G = (V,E) is the graph we want to color, S is the set of *all* maximal stable sets (i.e. independent sets)
(i.e. subsets of node of V which are not neighbors) (BTW, N(u) will denote the neighborhood of u)
$x_s$ is a binary variable and $x_s = 1$ means that all $u \in s$ receive the same color and $x_s = 0$ means nothing.

###### NOTE 0:
if $ x_s = 1$ and $x_s' = 1$, then they actually receive different colors, hence minimizing $\min\sum_{s \in S} x_s$
minimizes the number of colors. If a node is associated to two colors, I guess one can choose any (?)

Of course S is huge, so column generation.
Given the current master with stable sets $S'$:
- if it is infeasible (pricerfarkas): for each node u such that $u \notin s \forall s \in S'$ (this is the only way
for the LP to be infeasible), we find a maximal stable set in G which containt u and add it to $S'$
- if it is feasible (the pricing problem) (pricerredcost): given $\pi$, the dual solution, we solve
  $\max\{ \pi s : s\text{ is a stable set of G }\}$ and if it is larger than 1, we add s to S'

###### NOTE 1:
It seems that networkx doesn't provide functionality to solve the maximum weight independent set
        so we handle this by finding *all* independent sets and then dot product each of them with $\pi$ and
        select the maximum :-) (by them it is meant their indicator vector)

### TODO: Ab hier lesen
At some point we will have to branch. Now, given a fractional solution of master, consider $s \in S'
$ such that
$x_s$ is fractional. Its fractionality (i.e. $x_s < 1$) and feasibility implies that for every $u \in s$, there is
$s'$ (sorry for the $s'$ and $S'$) such that $u \in s'$ (and $x_s' > 0$). Also, sin both are maximal, then one *cannot*
be a subset of the other, which implies that there is $w \in V$ such that $w$ is in $s$ or $s'$, *but not* in both.
Hence, branching on "u and w have the same color" and "u and w have different colors" will:
- remove the current LP solution: Suppose s is the one that contains u and w (then s' contains only u). In the
  branch where SAME(u,w) (this means what it is suppose to mean), x_s' *has* to be 0 (and a lot of other guys, but
  that is beside the point).
  In the branch DIFF(u,w), now x_s *has* to be 0 (also a lot of other guys, but still beside the point)
  NOTE: this other guys thing is imposed in the propagation of the constraints which enforces the SAME and DIFF
- not remove any feasible solution: there are two types of coloring in the world, the ones in which u and w have
  the same color, and the ones in which u and w have different color

###### NOTE 2:
since u and w are in the same stable set (s or s'), then they are *not* neighbors. So imposing that they
        are different doesn't produce an infeasible subproblem (not sure why this note is important)

###### NOTE 3: 
the beauty of the branching rule is that both subproblems are of the same kind of problem as the original one,
        just on a different graph

Enforcing DIFF(u,w) amounts to add an edge between u and w, while enforcing
SAME(u,w) amounts to merge node u and w (i.e. create new node {u,w} (yes, the name is very unfortunate) and if
e = {v,u} or e = {v,w} existed in the original graph, then e = {v, {u,w}} exists in the "merged" graph)
The devil is in the details:
- Every node in the tree will have its own graph.
- Enforcing DIFF(u,w), in practice, means adding the edge *and* setting to 0 all x_s such that s contains both
  u and w (since such an x_s being equal to 1, means both variable have the same color; see NOTE 0).
  The edge will prevent the pricing problem to generate stable sets which contains both u and w
- Enforcing SAME(u,w), in practice, is a mess. One idea, as done in the C code, is to add enough edges so that
  N(u) = N(w). This is enough to ensure that every maximal stable set will either, have both, u and w or none of
  them. [A small proof: let s be a maximal stable set, if u \notin s, then there is v \in N(u) such that v \in s
  (otherwise, u could be added to s contradicting its maximality). Since v \in N(u) = N(w), then w cannot be in s]
  We use a different (hopefuly, more elegant; though I guess less efficient... but did we talk already about the
  brute force way of finding maximum weight independent sets?) approach here. We construct a "quotient graph", i.e.,
  a graph whose nodes are equivalence classes. The equivalence classes are the subsets of nodes which are equal (yes,
  we have been talking about enforcing that only u and w have to be equal, but imagine yourself now deep in the tree).

######   NOTE: 
We only use the quotient graph to search for stable sets. Once a stable set in the quotient graph is generated,
        we have to map it to a stable set of the original graph (where actually the constraints live), but this simple.
  Appart from that, we also have to set to 0 all x_s such that s contains u or w but not both.
###### NOTE 4:
the operations that enforce SAME and DIFF *preserve* maximal stable sets. So if we only add variables associated
        to maximal stable sets, then in every node of the tree the (valid) stable sets are going to be maximal

More details:
- We use a constraint handler to store the graph of each node. Given the graph of the parent node, generating the graphs
  of both childs, DIFF and SAME is done as follows:
  - DIFF(u,w): just add the corresponding edgea e={u,w}
  - SAME(u,w): build a new quotient graph, from the original graph. Let G be the original graph and H the parent's graph.
    Note that H is already a quotient graph of G. The equivalence relation used over the nodes to generate the child's graph
    is: node1 R node2 if and only if (node1 == u and node2 == w) or [node1]_H == [node2]_H
    where [v]_H is the equivalence class of v in H ([node1]_H == [node2]_H tests whether node1 and node2 are the same node in H)
- The graph that needs to be colored is given in some file. This graph gets pre-processed (see the C file for documentation)
  The "original graph" is the pre-processed graph.
END

## Bauen eines kleinen Modells mit Pricing
Der Pricer sollte keine Variablen hinzufügen, aber insgesamt das Programm laufen und der Pricer die richtigen Werte auslesen können

In [1]:
from pyscipopt import Model, Pricer, SCIP_RESULT, SCIP_STAGE

In [2]:
class VRPPricer(Pricer):
    def __init__(self, cons):
        self.cons = cons
        self.called = False
        
    def pricerfarkas(self):
        print("Farkas Pricing has been called.")
        return {'result':SCIP_RESULT.SUCCESS}

    def pricerredcost(self):
        if (not self.called) and self.model.getStage() >= SCIP_STAGE.SOLVING:  
            var = self.model.addVar(vtype="I",obj=1, pricedVar=True)
            cons = self.model.getTransformedCons(self.cons)
            self.model.addConsCoeff(cons, var, 2)
            print("Pricing added a variable\n")
            self.called = True
        print("Normal Pricing has been called.\n")
        return {'result':SCIP_RESULT.SUCCESS}

In [3]:
model = Model()
x = model.addVar("x",vtype="I",obj=1)
y = model.addVar("y",vtype="I",obj=1)
cons = model.addCons(x + y >= 1.5, modifiable=True)

pricer = VRPPricer(cons)
model.includePricer(pricer, "pricer","does pricing")
model.optimize()
model.printBestSol()
print("Hallo")
print(model.getBestSol())

Pricing added a variable
feasible solution found by trivial heuristic after 0.0 seconds, objective value 2.000000e+05
presolving:

Normal Pricing has been called.

Normal Pricing has been called.

Normal Pricing has been called.
presolving (1 rounds: 1 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 2 variables (0 bin, 2 int, 0 impl, 0 cont) and 1 constraints
      1 constraints of type <linear>
Presolving Time: 0.00
transformed 1/1 original solutions to the transformed problem space

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
i 0.0s|     1 |     0 |     0 |     - |  oneopt|   0 |   2 |   1 |   1 |   0 |  0 |   0 |   0 |      --      | 2.000000e+00 |    Inf | unknown
r 0.0s|     1 |     0 |     1 |     - |simplero|   0 |  